In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from imdb import IMDb
from imdb.Person import Person
from imdb.Company import Company
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

tmdb = pd.read_csv('tmdb-movies.csv')
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
id                      10866 non-null int64
imdb_id                 10856 non-null object
popularity              10866 non-null float64
budget                  10866 non-null int64
revenue                 10866 non-null int64
original_title          10866 non-null object
cast                    10790 non-null object
homepage                2936 non-null object
director                10822 non-null object
tagline                 8042 non-null object
keywords                9373 non-null object
overview                10862 non-null object
runtime                 10866 non-null int64
genres                  10843 non-null object
production_companies    9836 non-null object
release_date            10866 non-null object
vote_count              10866 non-null int64
vote_average            10866 non-null float64
release_year            10866 non-null int64
budget_adj              1

Questions:
(1)Does budget of a movie affect it's revenue?
(2)Does budget of a movies affect's it's popularity?
(3)Does revenue and popularity go hand in hand?
(4)Does runtime of a movie affect it's budget,revenue?
(5)Which genres are most popular with respect to budget,revenue and popularity?
(6)Which director has the most average popularity and revenue?
(7)Which actor has the most average popularity and revenue?
(8)Which production company has the most average popularity,budget and revenue?
(9)Which movies has most budget or most revenue or most popularity or most vote count or most revenue?
(10)Which movies has best ROI(i.e best profit percentage)
(11)Trend of average popularity,rating,budget,revenue,profit,voting,runtime over time?

In [13]:
tmdb.head(1)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,keywords,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,monster|dna|tyrannosaurus rex|velociraptor|island,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09


Assess:
(1)Cast is null in some movies
(2)Director is null in some movies
(3)Genres is null in some movies
(4)Production company is null in some movies
(5)Release date is string it should be date time
(6)Imdb id is null
(7)Budget is 0 and revenue is 0 for some movies
(8)Runtime is 0 for some movies
(9)50% data have budget and revenue as 0 need to investigate.(Missing values)
(10)Need to check count for null values(Hint:Use heat map of data lind)
(11)Tekken movie is duplicated and around 9 movies with imdb id as NAN(drop them)
(12)Recalcualte runtime for those whose runtime is > 184 i.e. 99 percentile

In [14]:
tmdb.sample(10)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,keywords,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
6188,33078,tt0090366,0.069492,0,0,A Zed & Two Noughts,Frances Barber|Joss Ackland|Brian Deacon|Eric ...,NaN,Peter Greenaway,NaN,NaN,"Oliver Deuce, a successful doctor, is shattere...",115,Comedy|Drama,NaN,10/4/85,10,5.5,1985,0.000000e+00,0.000000e+00
7475,7350,tt0825232,0.881034,45000000,175372502,The Bucket List,Jack Nicholson|Morgan Freeman|Sean Hayes|Bever...,http://www.warnerbros.com/bucket-list,Rob Reiner,"When he closed his eyes, his heart was opened",africa|brain tumor|wife husband relationship|c...,Corporate billionaire Edward and working class...,97,Comedy|Drama|Adventure,Warner Bros.|Storyline Entertainment|Two Ton F...,12/25/07,679,6.9,2007,4.732509e+07,1.844338e+08
10796,46878,tt0078350,0.205701,0,0,The Swarm,Michael Caine|Katharine Ross|Richard Widmark|R...,NaN,Irwin Allen,Monsters by the millions - and they're all for...,disaster|bee,Scientist Dr. Bradford Crane and army general ...,116,Action|Drama|Horror|Science Fiction,Warner Bros.,7/14/78,24,6.7,1978,0.000000e+00,0.000000e+00
2796,31002,tt0196158,0.272790,7000000,0,Ticker,Tom Sizemore|Steven Seagal|Dennis Hopper|Jaime...,NaN,Albert Pyun,NaN,NaN,"Ray Nettles, a detective of San Francisco, and...",92,Action|Adventure|Crime|Thriller,NaN,1/1/01,13,3.6,2001,8.620417e+06,0.000000e+00
5984,238407,tt3277552,0.043907,0,0,Aningaaq,Orto Ignatiussen|Lajla Lange|Maligiaq Fredeik|...,NaN,JonÃ¡s CuarÃ³n,NaN,NaN,"Aningaaq, an Inuit fisherman camping on the ic...",7,Drama,NaN,10/19/13,15,6.8,2013,0.000000e+00,0.000000e+00
475,301629,tt4000870,0.193063,0,0,Addicted to Fresno,Judy Greer|Aubrey Plaza|Natasha Lyonne|Clea Du...,NaN,Jamie Babbit,They always find a way to get off.,sister sister relationship|coverup|woman direc...,"Two co-dependent sisters, a recovering sex add...",85,Comedy,Andrea Sperling Productions|Lakeview Productio...,9/1/15,14,4.8,2015,0.000000e+00,0.000000e+00
5269,9574,tt0119137,0.960470,80000000,177977226,Flubber,Robin Williams|Marcia Gay Harden|Christopher M...,NaN,Les Mayfield,Catch it if you can!,wedding vows|inventor|slime|green|flight,"Professor Phillip Brainard, an absent minded p...",93,Comedy|Family|Science Fiction,Walt Disney Pictures|Great Oaks Entertainment,11/26/97,385,5.1,1997,1.086768e+08,2.417750e+08
555,341077,tt2911674,0.101155,0,0,Tiger House,Kaya Scodelario|Ed Skrein|Dougray Scott|Brando...,NaN,Tom Daley,"12 Hours, 4 Killers, 1 Way Out.",NaN,A young gymnast battles a group of bank robber...,80,Thriller|Drama|Action|Crime,Industrial Development Corporation of South Af...,8/24/15,44,5.3,2015,0.000000e+00,0.000000e+00
959,258086,tt2387408,0.336024,0,0,The Damned,Peter Facinelli|Sophia Myles|Nathalia Ramos|Ca...,NaN,VÃ­ctor GarcÃ­a,Heaven help us,NaN,"After suffering the recent loss of his wife, D...",87,Thriller|Horror|Mystery,Launchpad Productions|Bowery Hills Entertainme...,6/20/14,37,5.2,2014,0.000000e+00,0.000000e+00
8545,20539,tt0115988,0.490146,25000000,0,The Crucible,Daniel Day-Lewis|Winona Ryder|Paul Scofield|Jo...,NaN,Nicholas Hytner,Arthur Miller's timeless tale of truth on trial.,witch|massachusetts|passion|satan|curse,The movie is centered around the Salem Massach...,124,Drama|History,20th Century Fox,11/27/96,57,6.3,1996,3.475542e+07,0.000000e+00


In [15]:
tmdb.describe()

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,10866.000000,10866.000000,1.086600e+04,1.086600e+04,10866.000000,10866.000000,10866.000000,10866.000000,1.086600e+04,1.086600e+04
mean,66064.177434,0.646441,1.462570e+07,3.982332e+07,102.070863,217.389748,5.974922,2001.322658,1.755104e+07,5.136436e+07
std,92130.136561,1.000185,3.091321e+07,1.170035e+08,31.381405,575.619058,0.935142,12.812941,3.430616e+07,1.446325e+08
min,5.000000,0.000065,0.000000e+00,0.000000e+00,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,10596.250000,0.207583,0.000000e+00,0.000000e+00,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00
50%,20669.000000,0.383856,0.000000e+00,0.000000e+00,99.000000,38.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00
75%,75610.000000,0.713817,1.500000e+07,2.400000e+07,111.000000,145.750000,6.600000,2011.000000,2.085325e+07,3.369710e+07
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09


In [16]:
tmdb[tmdb.budget == 0].head(4)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,keywords,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
30,280996,tt3168230,3.927333,0,29355203,Mr. Holmes,Ian McKellen|Milo Parker|Laura Linney|Hattie M...,http://www.mrholmesfilm.com/,Bill Condon,The man behind the myth,london|detective|sherlock holmes,"The story is set in 1947, following a long-ret...",103,Mystery|Drama,BBC Films|See-Saw Films|FilmNation Entertainme...,6/19/15,425,6.4,2015,0.0,2.700677e+07
36,339527,tt1291570,3.358321,0,22354572,Solace,Abbie Cornish|Jeffrey Dean Morgan|Colin Farrel...,NaN,Afonso Poyart,"A serial killer who can see your future, a psy...",thriller|serial killer|psychic|fbi agent|intrigue,"A psychic doctor, John Clancy, works with an F...",101,Crime|Drama|Mystery,Eden Rock Media|FilmNation Entertainment|Flynn...,9/3/15,474,6.2,2015,0.0,2.056620e+07
72,284289,tt2911668,2.272044,0,45895,Beyond the Reach,Michael Douglas|Jeremy Irvine|Hanna Mangan Law...,NaN,Jean-Baptiste LÃ©onetti,NaN,desert|hunting,A high-rolling corporate shark and his impover...,95,Thriller,Furthur Films,4/17/15,81,5.5,2015,0.0,4.222338e+04
74,347096,tt3478232,2.165433,0,0,Mythica: The Darkspore,Melanie Stone|Kevin Sorbo|Adam Johnson|Jake St...,http://www.mythicamovie.com/#!blank/wufvh,Anne K. Black,NaN,sword|magic|sorcery|necromancer|woman director,When Teelaâ€™s sister is murdered and a powerf...,108,Action|Adventure|Fantasy,Arrowstorm Entertainment,6/24/15,27,5.1,2015,0.0,0.000000e+00


In [17]:
tmdb[tmdb.duplicated(['original_title'], keep=False)].sort_values(by='original_title').head(100)
#tmdb[tmdb.original_title.duplicated()]

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,keywords,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
5748,176068,tt2395385,0.336081,0,0,1,Rhys Wakefield|Logan Miller|Ashley Hinshaw|Nat...,NaN,Dennis Iliadis,Everyone wants one.,college|chaos|party|violence,Three college friends go to the biggest party ...,95,Thriller|Science Fiction,Process Productions,9/20/13,49,5.1,2013,0.000000e+00,0.000000e+00
5724,217316,tt2518788,0.364669,0,0,1,Michael Fassbender|Niki Lauda|Michael Schumach...,https://www.facebook.com/1thefilm,Paul Crowder,The True Story of the Drivers Who Risked Their...,speed|racing|formula 1|race car driver,Set in the golden era of Grand Prix Racing '1'...,112,Documentary,Exclusive Media Group|Universum Film (UFA)|Spi...,9/30/13,25,7.6,2013,0.000000e+00,0.000000e+00
6514,98622,tt0443424,0.128484,0,0,9,NaN,http://www.shaneacker.com,Shane Acker,NaN,soul|end of world,A rag doll fights a monster that has been stea...,11,Animation|Fantasy,NaN,4/21/05,49,7.3,2005,0.000000e+00,0.000000e+00
1400,12244,tt0472033,2.846465,30000000,48428063,9,Elijah Wood|John C. Reilly|Jennifer Connelly|C...,NaN,Shane Acker,When our world ended their mission began.,man vs machine|hope|post-apocalyptic|dystopia|...,"When 9 first comes to life, he finds himself i...",79,Action|Adventure|Animation|Science Fiction|Thr...,Focus Films|Starz Animation|Relativity Media|T...,8/19/09,851,6.5,2009,3.049201e+07,4.922230e+07
2489,16716,tt0216621,0.521573,0,0,A Christmas Carol,Patrick Stewart|Richard E. Grant|Joel Grey|Ian...,NaN,David Hugh Jones,NaN,holiday|scrooge|ghost,Scrooge is a miserly old businessman in 1840's...,95,Drama|Fantasy,Turner Network Television (TNT)|Hallmark Enter...,12/5/99,23,6.1,1999,0.000000e+00,0.000000e+00
1416,17979,tt1067106,1.714887,200000000,325233863,A Christmas Carol,Gary Oldman|Jim Carrey|Steve Valentine|Daryl S...,http://disney.go.com/disneypictures/achristmas...,Robert Zemeckis,Season's Greedings,holiday|victorian england|money|christmas eve|...,Miser Ebenezer Scrooge is awakened on Christma...,96,Animation|Drama,Walt Disney|ImageMovers,11/4/09,619,6.6,2009,2.032801e+08,3.305678e+08
7917,13189,tt0087056,0.542315,0,0,A Christmas Carol,George C. Scott|Roger Rees|David Warner|Susann...,NaN,Clive Donner,A new powerful presentation of the most loved ...,holiday|future|scrooge|christmas,An old bitter miser who makes excuses for his ...,100,TV Movie|Fantasy|Drama|Comedy|Family,Entertainment Partners Ltd.,12/17/84,29,6.6,1984,0.000000e+00,0.000000e+00
1994,23437,tt1179056,1.074308,35000000,115664037,A Nightmare on Elm Street,Jackie Earle Haley|Katie Cassidy|Kyle Gallner|...,http://www.nightmareonelmstreet.com/,Samuel Bayer,Never Sleep Again.,freddy krueger|reboot,The film that brings back horror icon Freddy K...,95,Horror|Mystery|Thriller,New Line Cinema|Platinum Dunes,4/30/10,423,5.5,2010,3.500000e+07,1.156640e+08
7891,377,tt0087800,1.331432,1800000,25504513,A Nightmare on Elm Street,John Saxon|Ronee Blakley|Heather Langenkamp|Am...,NaN,Wes Craven,"If Nancy Doesn't Wake Up Screaming, She Won't ...",child murderer|sleep|coffee|hat|nightmare,Teenagers in a small town are dropping like fl...,91,Horror,New Line Cinema|Smart Egg Pictures,11/15/84,625,7.1,1984,3.778276e+06,5.353504e+07
7039,8849,tt0375173,0.898601,60000000,13395939,Alfie,Jude Law|Marisa Tomei|Omar Epps|Jane Krakowski...,NaN,Charles Shyer,Meet a man who never met a woman he didn't love.,new york|cheating|flirt|lover|older woman sedu...,"In Manhattan, the British limousine driver Alf...",103,Comedy,Paramount Pictures|Patalex Productions,10/22/04,140,5.4,2004,6.926672e+07,1.546488e+07


In [18]:
tmdb.query('runtime>899')
tmdb['runtime'].quantile(0.99)

184.0

In [19]:
ia = IMDb()

# get a movie
movie = ia.get_movie('tt0061177'[2:],info=['main'])
moviekeys = movie.infoset2keys['main']

genrelist = ''
castlist=''
dirlist = ''
budget = 0
revenue = 0
runtime = 0
prodlist = ''
if 'runtimes' in moviekeys:
    runtime = int(movie['runtimes'][0])
if 'box office' in moviekeys:
    print(movie['box office'])
    if 'Budget' in movie['box office'] :
        stripbudget=movie['box office']['Budget'].strip()[1:]
        budget= int(stripbudget[:stripbudget.index(' ')].replace(',',''))
    if 'Cumulative Worldwide Gross' in movie['box office'] :
        striprevenue=movie['box office']['Cumulative Worldwide Gross'].strip()[1:]
        revenue = int(striprevenue[:striprevenue.index(' ')].replace(',',''))

if 'genres' in moviekeys:
    genrelist= '|'.join(movie['genres'])

if 'cast' in moviekeys:
    for cast in movie['cast']:
        castlist = castlist + '|' + cast['name'].strip()
    castlist = castlist.strip('|')

if 'directors' in moviekeys:
    for director in movie['directors']:
        dirlist = dirlist + '|' + director['name'].strip()
    dirlist = dirlist.strip('|')

if 'production companies' in moviekeys:
    for prodcomp in movie['production companies']:
        prodlist = prodlist + '|' + prodcomp['name'].strip()
    prolist = prodlist.strip('|')

print(budget)
print(revenue)
print(runtime)
print(genrelist)
print(castlist)
print(dirlist)
print(prodlist)
# print(movie['cast'][0]['name'])
# print(movie['directors'][0]['name'])
# print(movie['production companies'][0]['name'])
#print(movie['box office'])
#print(movie['box office']['Budget'])

0
0
80
Adventure|Comedy|Crime|Thriller
Woody Allen|The Lovin' Spoonful|Frank Buxton|Louise Lasser|Julie Bennett|Len Maxwell|Mickey Rose|Bryna Wilson|Tatsuya Mihashi|Mie Hama|Akiko Wakabayashi|Hideyo Amamoto|Steve Boone|Joe Butler|Susumu Kurobe|China Lee|Kumi Mizuno|Tadao Nakamaru|Tetsu Nakamura|Sachio Sakai|John Sebastian|Zal Yanovsky|Osman Yusuf
Woody Allen|Senkichi Taniguchi
|Toho Company|Benedict Pictures Corp.|National Recording Studios


In [20]:
samplestr = 'a,b,c'
if '|' in samplestr:
    print('Character found')

In [24]:
#Drop Null Id
tmdb.drop(tmdb[tmdb.imdb_id.isnull()].index,inplace = True)
tmdb[tmdb.imdb_id.isnull()]

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,keywords,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj


In [25]:
dirtyidlist = list(tmdb[(tmdb.budget == 0) | (tmdb.revenue == 0) | (tmdb.runtime == 0) | (tmdb.runtime > 184) | (tmdb.cast.isnull()) | (tmdb.director.isnull()) | (tmdb.genres.isnull()) | (tmdb.production_companies.isnull())].imdb_id.unique())

len(dirtyidlist)

7075

In [ ]:
# ia = IMDb()
# erroridlist = []
# df_list = []
# for num, imdbid in enumerate(dirtyidlist):
#     print("Imdb List {}: {}".format(num, imdbid))
#     try:
#         movie = ia.get_movie(imdbid[2:],info=['main'])
#         moviekeys = movie.infoset2keys['main']
#         genrelist = ''
#         castlist=''
#         dirlist = ''
#         budget = 0
#         revenue = 0
#         runtime = 0
#         prodlist = ''
#         if 'runtimes' in moviekeys:
#             runtime = int(movie['runtimes'][0])
#         if 'box office' in moviekeys:
#             if 'Budget' in movie['box office'] :
#                 stripbudget=movie['box office']['Budget'].strip()[1:]
#                 if ' ' in stripbudget:
#                     budget= int(stripbudget[:stripbudget.index(' ')].replace(',',''))
#                 else:
#                     budget= int(stripbudget.replace(',',''))
#             if 'Cumulative Worldwide Gross' in movie['box office'] :
#                 striprevenue=movie['box office']['Cumulative Worldwide Gross'].strip()[1:]
#                 if ' ' in striprevenue:
#                     revenue = int(striprevenue[:striprevenue.index(' ')].replace(',',''))
#                 else:
#                     revenue = int(striprevenue.replace(',',''))

#         if 'genres' in moviekeys:
#             genrelist= '|'.join(movie['genres'])

#         if 'cast' in moviekeys:
#             for cast in movie['cast']:
#                 castlist = castlist + '|' + cast['name'].strip()
#             castlist = castlist.strip('|')

#         if 'directors' in moviekeys:
#             for director in movie['directors']:
#                 dirlist = dirlist + '|' + director['name'].strip()
#             dirlist = dirlist.strip('|')

#         if 'production companies' in moviekeys:
#             for prodcomp in movie['production companies']:
#                 prodlist = prodlist + '|' + prodcomp['name'].strip()
#             prolist = prodlist.strip('|')
        
#         df_list.append({'imdbid': imdbid,
#                         'imdbbudget': budget,
#                         'imdbrevenue': revenue,
#                         'imdbruntime':runtime,
#                         'imdbgenres':genrelist,
#                         'imdbcast': castlist,
#                         'imdbdirector':dirlist,
#                         'imdbproduction_companies':prodlist
#                          })
            
#     except Exception as e:
#         print("Exception imdbid :" + imdbid + str(e))
#         erroridlist.append(imdbid)

# imdbmissingdata = pd.DataFrame(df_list, columns = ['imdbid', 'imdbbudget', 'imdbrevenue','imdbruntime','imdbgenres','imdbcast','imdbdirector','imdbproduction_companies'])
# imdbmissingdata.to_csv('missingmoviesdata.csv', index=False)    
# pd.read_csv('missingmoviesdata.csv')


In [64]:
ds = pd.read_csv('missingmoviesdata.csv')
#ds[~((ds.imdbbudget == 0) | (ds.imdbrevenue == 0))].info()
ds['imdb_id'] = ds.imdbid
del ds['imdbid']
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6690 entries, 0 to 6689
Data columns (total 8 columns):
imdbbudget                  6690 non-null int64
imdbrevenue                 6690 non-null int64
imdbruntime                 6690 non-null int64
imdbgenres                  6014 non-null object
imdbcast                    5989 non-null object
imdbdirector                5900 non-null object
imdbproduction_companies    5939 non-null object
imdb_id                     6690 non-null object
dtypes: int64(3), object(5)
memory usage: 418.2+ KB


In [65]:
#Fixing budget
budgetzerodf = tmdb[tmdb.budget == 0]
budgetnotzerodf = tmdb[~(tmdb.budget == 0)]
missingdatabudget = ds[~(ds.imdbbudget == 0)]
missingdatabudget
budgetjoin = pd.merge(budgetzerodf,missingdatabudget,on='imdb_id')
budgetjoin.budget = budgetjoin.imdbbudget
tmdbclean = budgetnotzerodf.append(budgetjoin[list(tmdb)])
tmdbclean.describe()

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,6561.000000,6561.000000,6.561000e+03,6.561000e+03,6561.000000,6561.000000,6561.000000,6561.000000,6.561000e+03,6.561000e+03
mean,46323.827313,0.862874,2.680855e+07,6.465598e+07,106.453285,334.271148,5.990184,2000.104252,2.906715e+07,8.285816e+07
std,73803.303291,1.213669,3.593077e+07,1.450871e+08,26.402611,715.501964,0.896634,12.530467,4.018046e+07,1.786287e+08
min,5.000000,0.001117,1.000000e+00,0.000000e+00,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,9383.000000,0.291704,5.000000e+06,0.000000e+00,93.000000,25.000000,5.400000,1993.000000,7.180884e+05,0.000000e+00
50%,13956.000000,0.528350,1.500000e+07,9.760104e+06,102.000000,79.000000,6.100000,2004.000000,1.381637e+07,1.297139e+07
75%,43566.000000,1.000194,3.300000e+07,6.354178e+07,116.000000,298.000000,6.600000,2010.000000,3.931458e+07,8.437300e+07
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,877.000000,9767.000000,8.500000,2015.000000,4.250000e+08,2.827124e+09


In [66]:
#Fixing revenue
revenuezerodf = tmdbclean[tmdbclean.revenue == 0]
revenuenotzerodf = tmdbclean[~(tmdbclean.revenue == 0)]
missingdatarevenue = ds[~(ds.imdbrevenue == 0)]
revenuejoin = pd.merge(revenuezerodf,missingdatarevenue,on='imdb_id')
revenuejoin.revenue = revenuejoin.imdbrevenue
tmdbclean = revenuenotzerodf.append(revenuejoin[list(tmdb)])
tmdbclean.describe()

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,4460.000000,4460.000000,4.460000e+03,4.460000e+03,4460.000000,4460.000000,4460.000000,4460.000000,4.460000e+03,4.460000e+03
mean,39878.191031,1.104298,3.489125e+07,9.780981e+07,108.734529,471.521525,6.137354,2000.652242,3.943264e+07,1.218907e+08
std,67391.993071,1.395857,4.032324e+07,1.668079e+08,21.574021,832.292707,0.800148,11.563505,4.407413e+07,2.053875e+08
min,5.000000,0.001117,1.000000e+00,2.000000e+00,15.000000,10.000000,2.100000,1960.000000,0.000000e+00,0.000000e+00
25%,7303.750000,0.427695,9.000000e+06,1.197872e+07,95.000000,58.000000,5.600000,1994.000000,8.403568e+06,1.119257e+07
50%,11454.500000,0.731675,2.100000e+07,3.959394e+07,105.000000,172.000000,6.200000,2004.000000,2.543922e+07,4.908394e+07
75%,37713.500000,1.275537,4.500000e+07,1.110554e+08,118.000000,493.000000,6.700000,2010.000000,5.519998e+07,1.418633e+08
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,705.000000,9767.000000,8.400000,2015.000000,4.250000e+08,2.827124e+09


In [67]:
#Fixing revenue
runtimezerodf = tmdbclean[(tmdbclean.runtime == 0) | (tmdbclean.runtime > 184)]
runtimenotzerodf = tmdbclean[~((tmdbclean.runtime == 0) | (tmdbclean.runtime > 184))]
missingdataruntime = ds
runtimejoin = pd.merge(runtimezerodf,missingdataruntime,on='imdb_id')
runtimejoin.runtime = runtimejoin.imdbruntime
tmdbclean = runtimenotzerodf.append(runtimejoin[list(tmdb)])
tmdbclean.describe()

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,4459.000000,4459.000000,4.459000e+03,4.459000e+03,4459.000000,4459.000000,4459.000000,4459.000000,4.459000e+03,4.459000e+03
mean,39883.978246,1.104458,3.489720e+07,9.782883e+07,108.643866,471.616506,6.136981,2000.650594,3.943958e+07,1.219151e+08
std,67398.442814,1.395972,4.032581e+07,1.668217e+08,20.930092,832.361875,0.799850,11.564278,4.407664e+07,2.054040e+08
min,5.000000,0.001117,1.000000e+00,2.000000e+00,0.000000,10.000000,2.100000,1960.000000,0.000000e+00,0.000000e+00
25%,7303.500000,0.427821,9.000000e+06,1.196898e+07,95.000000,58.000000,5.600000,1994.000000,8.393787e+06,1.118692e+07
50%,11454.000000,0.731802,2.100000e+07,3.967316e+07,105.000000,172.000000,6.200000,2004.000000,2.545749e+07,4.909428e+07
75%,37717.000000,1.275693,4.500000e+07,1.111140e+08,118.000000,493.000000,6.700000,2010.000000,5.519998e+07,1.418805e+08
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,594.000000,9767.000000,8.400000,2015.000000,4.250000e+08,2.827124e+09


In [68]:
#Fixing cast,genre,director,production company is null
nulldf = tmdbclean[(tmdbclean.cast.isnull()) | (tmdbclean.director.isnull()) | (tmdbclean.genres.isnull()) | (tmdbclean.production_companies.isnull())]
notnulldf = tmdbclean[~((tmdbclean.cast.isnull()) | (tmdbclean.director.isnull()) | (tmdbclean.genres.isnull()) | (tmdbclean.production_companies.isnull()))]
missingdata = ds
nulljoin = pd.merge(nulldf,missingdata,on='imdb_id')
nulljoin.cast = nulljoin.imdbcast
nulljoin.director = nulljoin.imdbdirector
nulljoin.genres = nulljoin.imdbgenres
nulljoin.production_companies = nulljoin.imdbproduction_companies
tmdbclean = notnulldf.append(nulljoin[list(tmdb)])
tmdbclean.describe()

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,4454.000000,4454.000000,4.454000e+03,4.454000e+03,4454.000000,4454.000000,4454.000000,4454.000000,4.454000e+03,4.454000e+03
mean,39895.186125,1.105504,3.492907e+07,9.792017e+07,108.618096,472.114055,6.136798,2000.643018,3.947622e+07,1.220332e+08
std,67431.919078,1.396404,4.033708e+07,1.668901e+08,20.898036,832.696259,0.800115,11.568428,4.408767e+07,2.054866e+08
min,5.000000,0.001117,1.000000e+00,2.000000e+00,0.000000,10.000000,2.100000,1960.000000,0.000000e+00,0.000000e+00
25%,7303.250000,0.429133,9.000000e+06,1.200000e+07,95.000000,58.000000,5.600000,1994.000000,8.413349e+06,1.124848e+07
50%,11452.500000,0.733621,2.100000e+07,3.969813e+07,105.000000,172.000000,6.200000,2004.000000,2.554816e+07,4.921709e+07
75%,37704.000000,1.277284,4.500000e+07,1.113134e+08,118.000000,493.000000,6.700000,2010.000000,5.524176e+07,1.420315e+08
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,594.000000,9767.000000,8.400000,2015.000000,4.250000e+08,2.827124e+09


In [72]:
tmdbclean.info()
tmdbclean.genres = tmdbclean['genres'].str.lower()
tmdbclean.cast = tmdbclean['cast'].str.lower()
tmdbclean.director = tmdbclean['director'].str.lower()
tmdbclean.production_companies = tmdbclean['production_companies'].str.lower()
tmdbclean.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4454 entries, 0 to 61
Data columns (total 21 columns):
id                      4454 non-null int64
imdb_id                 4454 non-null object
popularity              4454 non-null float64
budget                  4454 non-null int64
revenue                 4454 non-null int64
original_title          4454 non-null object
cast                    4451 non-null object
homepage                1520 non-null object
director                4450 non-null object
tagline                 4077 non-null object
keywords                4272 non-null object
overview                4454 non-null object
runtime                 4454 non-null int64
genres                  4451 non-null object
production_companies    4451 non-null object
release_date            4454 non-null object
vote_count              4454 non-null int64
vote_average            4454 non-null float64
release_year            4454 non-null int64
budget_adj              4454 non-null float6

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,keywords,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
894,11527,tt0082348,0.995357,11000000,34967437,Excalibur,nigel terry|helen mirren|nicholas clay|cherie ...,NaN,john boorman,Forged by a god. Foretold by a wizard. Found b...,holy grail|camelot|round table|wizardry|merlin,The myth of King Arthur brought to the screen....,140,adventure,orion pictures|warner bros.,4/10/81,107,6.5,1981,0.000000e+00,8.385859e+07
3964,1430,tt0310793,1.005772,4000000,35564473,Bowling for Columbine,michael moore|george h. w. bush|george w. bush...,http://www.bowlingforcolumbine.com/index.php,michael moore,One nation under the gun.,usa|canada|gun|homicide|to shoot dead,"Why do 11,000 people die in America each year ...",120,documentary,alliance atlantis communications|time film- un...,10/9/02,235,7.2,2002,4.849046e+06,4.311344e+07
8682,1597,tt0212338,1.207663,55000000,330444045,Meet the Parents,ben stiller|robert de niro|teri polo|blythe da...,NaN,jay roach,First comes love. Then come the interrogation.,cia|airport|cat|jew|orderly,"Greg Focker is ready to marry his girlfriend, ...",108,comedy|romance,dreamworks skg|universal pictures|tribeca prod...,10/5/00,957,6.5,2000,6.964607e+07,4.184387e+08
8105,11017,tt0112508,0.938457,10000000,26488734,Billy Madison,adam sandler|darren mcgavin|bridgette wilson|b...,NaN,tamra davis,Billy is going back to school... Way back.,woman director,Billy Madison is the 27 year-old son of Bryan ...,89,comedy,jack giarraputo productions,2/10/95,237,6.2,1995,1.430967e+07,3.790450e+07
8726,11973,tt0146309,0.627609,80000000,34566746,Thirteen Days,kevin costner|bruce greenwood|steven culp|dyla...,NaN,roger donaldson,You'll never believe how close we came,u.s. president|atomic bomb|john f. kennedy|kub...,"Dramatisation of the Cuban Missile Crisis, the...",145,drama|thriller,new line cinema|tig productions,12/24/00,94,6.6,2000,1.013034e+08,4.377160e+07


In [82]:
tmdbclean[tmdbclean.duplicated(['imdb_id'], keep=False)] #tt0411951
tmdbclean.drop_duplicates(subset ="imdb_id", keep = "first",inplace =True)
tmdbclean[tmdbclean.imdb_id == "tt0411951"]
tmdbclean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4453 entries, 0 to 61
Data columns (total 21 columns):
id                      4453 non-null int64
imdb_id                 4453 non-null object
popularity              4453 non-null float64
budget                  4453 non-null int64
revenue                 4453 non-null int64
original_title          4453 non-null object
cast                    4450 non-null object
homepage                1520 non-null object
director                4449 non-null object
tagline                 4076 non-null object
keywords                4271 non-null object
overview                4453 non-null object
runtime                 4453 non-null int64
genres                  4450 non-null object
production_companies    4450 non-null object
release_date            4453 non-null object
vote_count              4453 non-null int64
vote_average            4453 non-null float64
release_year            4453 non-null int64
budget_adj              4453 non-null float6

In [89]:
#Drop invalid columns
tmdbclean.drop(tmdbclean[(tmdbclean.budget == 0) | (tmdbclean.revenue == 0) | (tmdbclean.runtime == 0) | (tmdbclean.cast.isnull()) | (tmdbclean.director.isnull()) | (tmdbclean.genres.isnull()) | (tmdbclean.production_companies.isnull())].index,inplace = True)
tmdbclean.info()
tmdbclean.describe()
tmdbclean.sample(5)
tmdbclean.to_csv('tmdbclean.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4429 entries, 3 to 61
Data columns (total 21 columns):
id                      4429 non-null int64
imdb_id                 4429 non-null object
popularity              4429 non-null float64
budget                  4429 non-null int64
revenue                 4429 non-null int64
original_title          4429 non-null object
cast                    4429 non-null object
homepage                1506 non-null object
director                4429 non-null object
tagline                 4055 non-null object
keywords                4248 non-null object
overview                4429 non-null object
runtime                 4429 non-null int64
genres                  4429 non-null object
production_companies    4429 non-null object
release_date            4429 non-null object
vote_count              4429 non-null int64
vote_average            4429 non-null float64
release_year            4429 non-null int64
budget_adj              4429 non-null float6